In [1]:
import numpy as np
np.random.seed(10)
import tensorflow as tf
tf.random.set_seed(10)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import matplotlib.pyplot as plt

2022-10-15 12:15:06.662123: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-15 12:15:06.827665: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-15 12:15:13.614723: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-15 12:15:13.614801: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [15]:
from pickle import dump

In [2]:
# In physical space use 'raw_train.reshape(-1,121,281)' to visualize original
num_snapshots = 12564
directory = '/lcrc/project/AIEADA-2/era5_data/full_data/snapshots/'
raw_train_z500 = np.load(directory + 'train_z500_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1) # Subsample to fit on GPU
raw_train_u250 = np.load(directory + 'train_u250_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)
raw_train_v250 = np.load(directory + 'train_v250_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)
raw_train_u850 = np.load(directory + 'train_u850_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)
raw_train_v850 = np.load(directory + 'train_v850_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)
raw_train_t250 = np.load(directory + 'train_t250_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)
raw_train_t850 = np.load(directory + 'train_t850_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)
raw_train_blh = np.load(directory + 'train_blh_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)
raw_train_tcwv = np.load(directory + 'train_tcwv_snap.npy').reshape(-1,121,281)[:,::2,::4].reshape(num_snapshots,-1)

var_list = [raw_train_z500,
            raw_train_u250,raw_train_v250,raw_train_t250,
            raw_train_u850,raw_train_v850,raw_train_t850,
            raw_train_blh,raw_train_tcwv]

scaler_list = []
for i in range(len(var_list)):
    scaler = MinMaxScaler()
    var_list[i] = scaler.fit_transform(var_list[i])
    scaler_list.append(scaler)
    
# Free memory
del raw_train_z500,raw_train_u250,raw_train_v250,raw_train_t250,raw_train_u850,raw_train_v850,raw_train_t850,raw_train_blh,raw_train_tcwv

In [3]:
total_bytes = 0
for array in var_list:
    total_bytes += array.nbytes
onegb = 1000000000
print(total_bytes/onegb)

1.958928624


In [4]:
raw_train = np.concatenate(var_list,axis=-1)
# Free memory
del var_list

In [5]:
print(raw_train.shape)

(12564, 38979)


In [6]:
input_window = 14
output_window = 7

train_inputs = []
train_outputs = []

i = 0
while i < raw_train.shape[0] - input_window - output_window:
    train_inputs.append(raw_train[i:i+input_window])
    train_outputs.append(raw_train[i+input_window:i+input_window+output_window])
    i = i + 1
    
del raw_train
    
train_inputs = np.asarray(train_inputs)
train_outputs = np.asarray(train_outputs)

In [7]:
gb_train_inputs = train_inputs.nbytes/onegb
gb_train_outputs = train_outputs.nbytes/onegb
print(gb_train_inputs)
print(train_inputs.shape)
print(gb_train_outputs)
print(train_outputs.shape)

27.379161432
(12543, 14, 38979)
13.689580716
(12543, 7, 38979)


## This is a lot of data to fit into memory, so let's save it to files and later use TensorFlow data pipelines to load it as needed

In [13]:
train_dir = directory + "/split_examples/train_data/"
for i in range(train_inputs.shape[0]):
    example_input = train_inputs[i,:,:]
    example_output = train_outputs[i,:,:]
    example = np.concatenate([example_input, example_output],axis=0)
    np.save(train_dir + "example%d.npy" % i, example)

In [16]:
dump(scaler_list, open(directory + '/split_examples/scalers.pkl', 'wb'))